https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

25/03/10 17:29:11 WARN Utils: Your hostname, bigdata resolves to a loopback address: 127.0.1.1; using 10.3.133.231 instead (on interface ens3)
25/03/10 17:29:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/03/10 17:29:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/10 17:29:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9998) \
    .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

25/03/10 17:29:15 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


Next

goto terminal
run 
nc -lk 9998


In [3]:
 # Start running the query that prints the running counts to the console
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()


25/03/10 17:29:16 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-bc5ecbc1-09ea-4a0d-83c4-7215c1728f6e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hello|    2|
+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hello|    2|
|  abc|    1|
|  hey|    1|
| test|    1|
+-----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hello|    2|
|  abc|    1|
|  abb|    1|
|  hey|    1|
| test|    1|
+-----+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|  ccd|    1|
|hello|    2|
|  eed|    1|
|  abc|    1|
|  abb|    1|
|  hey|    1|
| test|    1|
+-----+-----+



-------------------------------------------
Batch: 5
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
| aaab|    1|
|  ccd|    2|
|hello|    2|
|  eed|    1|
|  abc|    1|
|  abb|    1|
|  hey|    1|
| test|    1|
+-----+-----+



-------------------------------------------
Batch: 6
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
| aaab|    1|
|  ccd|    2|
|hello|    2|
|  eed|    1|
|  abc|    1|
|  abb|    1|
|  aad|    1|
|  hey|    1|
| test|    1|
|   aa|    1|
+-----+-----+



KeyboardInterrupt: 

next at the nc terminal
type:

hello
hey
abc
test
abb
ccd
eed
aaab
ccd
aa
aad